In [2]:
import random
import twitter
import emoji
import gensim
import unicodedata
import html
from keras.preprocessing.text import text_to_word_sequence
import re

Using TensorFlow backend.


In [3]:
# Fill these in!

CONSUMER_KEY = 'xbMuxcJpRTiVGt2C2EYnA'
CONSUMER_SECRET = '2DbQTsvIptkPTdaUcos8DDvQH9fzO0hNjJpUT2uVzQ'
ACCESS_TOKEN = '7319442-EDm4CPxL7W4KkZcGWRMJNVHp88W5OH9vgblu898fg'
ACCESS_SECRET = '5ZxJSbqXhG7uhgXzTFWf9XhkfsxxinlPRXyDTzbA9w'

In [4]:
auth=twitter.OAuth(
    consumer_key=CONSUMER_KEY,
    consumer_secret=CONSUMER_SECRET,
    token=ACCESS_TOKEN,
    token_secret=ACCESS_SECRET,
)

status_stream = twitter.TwitterStream(auth=auth).statuses
next(status_stream.sample()).keys()

dict_keys(['filter_level', 'in_reply_to_screen_name', 'user', 'coordinates', 'id_str', 'favorited', 'truncated', 'contributors', 'in_reply_to_status_id_str', 'source', 'retweet_count', 'in_reply_to_status_id', 'entities', 'extended_entities', 'lang', 'favorite_count', 'retweeted_status', 'reply_count', 'id', 'place', 'text', 'is_quote_status', 'quote_count', 'timestamp_ms', 'in_reply_to_user_id_str', 'geo', 'possibly_sensitive', 'created_at', 'retweeted', 'in_reply_to_user_id'])

In [5]:
RE_URL = re.compile(r'(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?')
RE_WHITESPACE = re.compile(r'\s+')

def strip_accents(s):
     return ''.join(c for c in unicodedata.normalize('NFD', s)
                    if unicodedata.category(c) != 'Mn')

class TokensYielder(object):
    def __init__(self, tweet_count, stream):
        self.tweet_count = tweet_count
        self.stream = stream

    def __iter__(self):
        count = self.tweet_count
        for tweet in self.stream:
            if tweet.get('lang') != 'en':
                continue
            text = tweet['text']
            text = html.unescape(text)
            text = RE_WHITESPACE.sub(' ', text)
            text = RE_URL.sub(' ', text)
            text = strip_accents(text)
            text = ''.join(ch for ch in text if ord(ch) < 128)
            if text.startswith('RT '):
                text = text[3:]
            text = text.strip()
            if text:
                yield text_to_word_sequence(text)
                count -= 1
                if count <= 0:
                    break

for t in TokensYielder(3, twitter.TwitterStream(auth=auth).statuses.sample()):
    print(t)

!
['narendramodi', 'best', 'wishes', 'to', 'our', 'former', 'prime', 'minister', 'shri', 'hd', 'deve', 'gowda', 'ji', 'on', 'his', 'birthday', 'i', 'pray', 'for', 'his', 'long', 'and', 'healthy', 'life']
['ty', 'lightwood', 'aristotle', 'and', 'dante', 'discover', 'the', 'secrets', 'of', 'the', 'universe', '2019']
['goaaaaaalllll', 'gabrieeeell', 'jesuss']


In [7]:
tweets = list(TokensYielder(70000, twitter.TwitterStream(auth=auth).statuses.sample()))

!


In [8]:
model = gensim.models.Word2Vec(tweets, 
                               workers=5,
                               min_count=2,
                              )
model.save('zoo/07/twitter_stream_w2v.model')

In [9]:
model.wv.most_similar(positive=['love'], topn=5)

/home/douwe/checkout/deep_learning_cookbook/venv3/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('miss', 0.822679877281189),
 ('hope', 0.8068050146102905),
 ('loved', 0.8038904666900635),
 ('appreciate', 0.8034697771072388),
 ('ramblingsloa', 0.8009338974952698)]